In [15]:
import functools

In [69]:
def check_arguments(*types):
    def my_decorator(f):
        @functools.wraps(f)
        def wrapper_function(*args):
            if types is not None:
                if args.__len__() < types.__len__():
                    raise TypeError

                for pos in range(types.__len__()):
                    if not isinstance(args[pos], types[pos]):
                        raise TypeError
            return f(*args)
        
        return wrapper_function
    
    return my_decorator

@check_arguments(int, int)
def test_function(x : int, y : int ):
    print(x)
    print(y)

In [70]:
test_function('abc', 6)

TypeError: 

In [242]:
def substitutive(f):
    def wrapper_function(*args, **kwargs):
        #print("Func: ", f.__name__)
        #print("Args: ", args, args.__len__())
        #print("Kwargs: ", kwargs)
        #print("All_args: ", arg_names)

        for pos in range(args.__len__()):
            if arg_names[pos] in kwargs.keys():
                raise TypeError

        needed_args = []
        for pos in range(args.__len__(), arg_names.__len__()):
            if arg_names[pos] not in kwargs.keys():
                needed_args.append(arg_names[pos])

        def f1(*extra_args):
            if extra_args.__len__() != needed_args.__len__():
                raise TypeError

            list_args = list(args)
            for item in extra_args:
                list_args.append(item)

            return f(*tuple(list_args), **kwargs)

        return f1
    
    return wrapper_function

@substitutive
def test_function(a, b, c, d):
    return a + b + c + d

In [313]:
from inspect import signature
import numpy as np


def substitutive(f):
    
    sig = signature(f)
    
    args_list = []
    for item in sig.parameters:
        args_list.append(item)
        
    print(args_list)
    
    args_am = args_list.__len__()
    args_accordings = np.array(args_list)
    print(args_accordings)
    
    def wrapper_function(*args, **kwargs):
        
        args_names_used = []
        
        for pos in range(args.__len__()):
            # checking for double initialisation,
            # keyword already cannot be set twice
            if args_list[pos] in kwargs.keys():
                raise TypeError
        
        for pos in range(args.__len__()):
            # replacing args
            args_accordings[pos] = args[pos]
            args_names_used.append(args_list[pos])
            
        args_used = args.__len__()
        for key in kwargs.keys():
            # replacing kwargs
            if key in args_list:
                args_accordings[args_list.index(key)] = kwargs[key]
                args_used += 1
                args_names_used.append(key)
            
        
        print(args_accordings)
        print(args_used)
        
        args_not_used = [i for i in args_list if i not in args_names_used]
        
        print(args_not_used)
        
        def f1(*extra_args, **extra_kwargs):
            pass
        
        return f1
    
    return wrapper_function
            
            
@substitutive
def test_function(a, b, c, d):
    return a + b + c + d

['a', 'b', 'c', 'd']
['a' 'b' 'c' 'd']


In [612]:
from inspect import getargspec

def substitutive(f):
    
    args_list = inspect.getargspec(f).args
    print(dir(f.__code__))
    print(f.__dir__())
        
    def wrapper_function(*args, **kwargs):
        #print("Func: ", f.__name__)
        #print("Args: ", args, args.__len__())
        #print("Kwargs: ", kwargs)
        #print("All_args: ", arg_names)

        for pos in range(args.__len__()):
            if args_list[pos] in kwargs.keys():
                raise TypeError

        needed_args = []
        for pos in range(args.__len__(), args_list.__len__()):
            if args_list[pos] not in kwargs.keys():
                needed_args.append(args_list[pos])
            
        
        if needed_args.__len__() == 0:
            return f(*args, **kwargs)

        def f1(*extra_args, **extra_kwargs):
            
            if extra_args.__len__() + \
               extra_kwargs.__len__() != needed_args.__len__():
                raise TypeError
                
            for item in extra_args:
                if item in extra_kwargs.keys():
                    raise TypeError
                    
            args_list_now = list(args)
            
            for item in extra_args:
                args_list_now.append(item)
                
            for key in kwargs.keys():
                extra_kwargs[key] = kwargs[key]
                
            print(args_list_now)
            print(args_list)
            
            for pos in range(args_list_now.__len__()):
                extra_kwargs[args_list[pos]] = args_list_now[pos]
                
            return f(**extra_kwargs)
            
        return f1
    
    print(f.__code__.co_argcount)
    
    return wrapper_function

@substitutive
def test_function(a, b, c, d):
    e = 1
    return a + b + c + d

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'co_argcount', 'co_cellvars', 'co_code', 'co_consts', 'co_filename', 'co_firstlineno', 'co_flags', 'co_freevars', 'co_kwonlyargcount', 'co_lnotab', 'co_name', 'co_names', 'co_nlocals', 'co_stacksize', 'co_varnames']
['__repr__', '__call__', '__get__', '__new__', '__closure__', '__doc__', '__globals__', '__module__', '__code__', '__defaults__', '__kwdefaults__', '__annotations__', '__dict__', '__name__', '__qualname__', '__hash__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__init__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']
4


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  """


In [651]:
def update_wrapper(func1, func2):
    func1.__module__ = func2.__module__
    func1.__name__ = func2.__name__
    func1.__doc__ = func2.__doc__
    return func1

def sub(f):
    
    args_am = f.__code__.co_argcount
    args_saved = []

    def wrapper_function(*args, **kwargs):
        
        print('here_outer')
        first_run = True
        def inner_wrapper(*args, **kwargs):
            
            print('here_inner')
            inner_args = list()
            
            if first_run:
                inner_args = args_saved.copy()
                first_run = False
                
            print("here")
            if args.__len__() > 0:
                for item in args:
                    inner_args.append(item)
            
            inner_am = inner_args.__len__()
            
            if inner_am == args_am:
                return f(*inner_args)
            elif inner_am > args_am:
                raise TypeError
                
            return update_wrapper(inner_wrapper, f)
        
        if args.__len__() > 0:
            for item in args:
                args_saved.append(item)
                
        used_am = args_saved.__len__()
        
        if used_am == args_am:
            return f(*args_saved)
        elif used_am > args_am:
            raise TypeError
            
        # if we have to add more args
        return inner_wrapper
    
    return update_wrapper(wrapper_function, f)
        

@sub
def test_f(a, b, c, d):
    return a + b + c + d

In [652]:
@sub
def f(x, y, z):
    "Some doc"
    print(x, y, z)

g = f(1, 2)
g(3)
g(4)
g(5)

here_outer
here_inner


UnboundLocalError: local variable 'first_run' referenced before assignment

In [360]:
hmm = test_function(1, 2)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  """


In [361]:
hmm(3)

IndexError: list index out of range

In [563]:
def substit(f):

    args_list = inspect.getfullargspec(f).args
    args_used = []
    
    args_new = []
    #print('here_1')
    
    def f2(*extra_args_2):
        args_new.clear()
        #print("here_3")

        def f1(*extra_args_1):
            #print('here_4')

            for item in extra_args_1:
                args_new.append(item)

            if args_new.__len__() + args_used.__len__() == args_list.__len__():
                res_args = []
                for item in args_used:
                    res_args.append(item)

                for item in args_new:
                    res_args.append(item)
                args_new.clear()
                return f(*res_args)
            elif args_used.__len__() + args_new.__len__() > args_list.__len__():
                raise TypeError
                
            return f1

        return f1(*extra_args_2)
      
    
    def wrapper_function(*args, **kwargs):
        
        if args.__len__() == args_list.__len__():
            return f(*args)
            
        args_used.clear()
        args_new.clear()
        
        for item in args:
            args_used.append(item)
            
        #print(args_used)
        
        return f2
    
    return wrapper_function
    
@substit
def f(x, y, z):
    print(x, y, z)

In [564]:
try:
    f(1, 2, 3)
except Exception as e:
    print(type(e).__name__)

1 2 3


In [546]:
b = a(2)

In [547]:
b(3)

1 2 3


In [354]:
@substitutive
def f(x, y, z):
    print(x, y, z)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  """


In [355]:
f(1, 2, 3)

[]
1 2 3


In [346]:
hmm(4, 4, 5)

[1, 4, 4, 5]
['a', 'b', 'c', 'd']


14

In [236]:
from inspect import signature
import inspect

In [250]:
sig = signature(test_function)

In [251]:
str(sig.parameters)

'OrderedDict([(\'args\', <Parameter "*args">), (\'kwargs\', <Parameter "**kwargs">)])'

In [252]:
@substitutive
def test_function(a, b, c, d):
    return a + b + c + d

inspect.getfullargspec(test_function)

FullArgSpec(args=[], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [246]:
test_function()

NameError: name 'arg_names' is not defined

In [269]:
test = {}
test['1'] = 1
test['2'] = 2

In [272]:
test['3'] = 123

In [273]:
test.__len__()

3

In [277]:
for key in test.keys():
    print(key, test[key])

1 1
2 2
3 123


In [283]:
import numpy as np

In [291]:
ar = np.array(['a', 'b', 'c'])

In [302]:
ar.tolist().index('a')

0

In [295]:
np.where(ar == 'a')[0][0]

0

In [306]:
l1 = ['a', 'b', 'c']
l2 = ['a', 'b']

[i for i in l1 + l2 if i not in l1 or i not in l2]

['c', 'd']